In [1]:
# Import the necessary modules
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# Define the hyperparameters
batch_size = 128
num_epochs = 10
temperature = 10 # A scaling factor for the soft targets
alpha = 0.9 # A weighting factor for the soft loss

# Load the CIFAR-10 dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# Define the teacher model (a deep convolutional neural network)
class Teacher(nn.Module):
    def __init__(self):
        super(Teacher, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, 3, padding=1)
        self.conv2 = nn.Conv2d(64, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv4 = nn.Conv2d(128, 128, 3, padding=1)
        self.fc1 = nn.Linear(128 * 8 * 8, 256)
        self.fc2 = nn.Linear(256, 256)
        self.fc3 = nn.Linear(256, 10)

    def forward(self, x):
        x = self.pool(nn.ReLU()(self.conv2(nn.ReLU()(self.conv1(x)))))
        x = self.pool(nn.ReLU()(self.conv4(nn.ReLU()(self.conv3(x)))))
        x = x.view(-1, 128 * 8 * 8)
        x = nn.ReLU()(self.fc1(x))
        x = nn.ReLU()(self.fc2(x))
        x = self.fc3(x)
        return x

# Define the student model (a shallow fully-connected neural network)
class Student(nn.Module):
    def __init__(self):
        super(Student, self).__init__()
        self.fc1 = nn.Linear(3 * 32 * 32, 256)
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = x.view(-1, 3 * 32 * 32)
        x = nn.ReLU()(self.fc1(x))
        x = self.fc2(x)
        return x

# Instantiate the models and move them to the device (CPU or GPU)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
teacher = Teacher().to(device)
student = Student().to(device)

# Define the loss function and the optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(student.parameters(), lr=0.01, momentum=0.9)

# Train the teacher model on the original hard targets
teacher.train()
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = teacher(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 200 == 199:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0
print('Finished training the teacher model')

# Evaluate the teacher model on the test set
teacher.eval()
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = teacher(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print('Accuracy of the teacher model on the test set: %d %%' % (100 * correct / total))

# Train the student model on the soft targets from the teacher model
student.train()
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = student(inputs)
        with torch.no_grad():
            teacher_outputs = teacher(inputs)
        # Compute the soft loss and the hard loss
        soft_loss = nn.KLDivLoss()(nn.LogSoftmax(dim=1)(outputs / temperature), nn.Softmax(dim=1)(teacher_outputs / temperature))
        hard_loss = criterion(outputs, labels)
        # Combine the soft loss and the hard loss with a weighting factor
        loss = alpha * soft_loss + (1 - alpha) * hard_loss
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 200 == 199:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0
print('Finished training the student model')

# Evaluate the student model on the test set
student.eval()
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = student(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print('Accuracy of the student model on the test set: %d %%' % (100 * correct / total))


100%|██████████| 170498071/170498071 [00:10<00:00, 16449219.20it/s]


Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified
[1,   200] loss: 2.303
